### 03. RAG - Retrieval-Augmented Generation

- skicka med information som man önskar att LLM använder för att generera ett svar
- kan vara egen data som modell ej är tränad på eller data som är nyare än modellens träningsdata


In [2]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel

from google import genai # client for google gemini api

In [3]:
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY") # load the api key from another file (.env)

In [4]:
client = genai.Client(api_key=API_KEY) # create an instance of the client

In [19]:
# load data to be provided to the model
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"who won the masters in 2025?"
)

# print the response
print(response.candidates[0].content.parts[0].text)

As an AI, I do not have access to real-time information, including sports results that haven't happened yet. The Masters Tournament 2025 has not taken place, so there is no winner yet. It will be held in April 2025.



In [22]:
# provide information that can be used by the model

winners = None
with open("data/masters-winners.csv", encoding="utf-8") as f:
    winners = f.read()


In [23]:
# load data to be provided to the model
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"feel free to use the following information to answer the question: {winners}, who won the masters in 2025? "
)

# print the response
print(response.candidates[0].content.parts[0].text)


Rory McIlroy won the Masters in 2025.

